In [1]:
import numpy as np
import pandas as pd
from numpy import dot
from numpy.linalg import norm
from sentence_transformers import SentenceTransformer
import pymysql

In [2]:
model = SentenceTransformer('sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens')

In [3]:
# mysql 연동
conn = pymysql.connect(host='127.0.0.1'
                , port=3306
                , user = 'root'
                , password='root1234'
                , db = 'finaldb'
                , charset='utf8'
                ) 

cur = conn.cursor()

In [7]:
# 가장 유사한 플레이리스트 추출
def cos_sim(A, B):
  return dot(A, B)/(norm(A)*norm(B))

In [341]:
# input
new_title = "비 오는 날 카페에서 창 밖을 바라보며"

In [342]:
cur = conn.cursor()
new_embedding = model.encode(new_title)



In [343]:
sql = 'SELECT * FROM ply_title_embedding'
ply_title_embedding = pd.read_sql(sql, conn)

C:\Users\k9942\AppData\Local\Temp\ipykernel_23216\837042529.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  ply_title_embedding = pd.read_sql(sql, conn)


In [344]:
# str -> embedding(array)
ply_title_embedding['ply_title_emd'] = ply_title_embedding.apply(lambda x: np.fromstring(x['ply_title_emd'],dtype=float, sep=','), axis=1)

# cosine similarity 계산
ply_title_embedding['cos_sim'] = ply_title_embedding.apply(lambda x: cos_sim(x['ply_title_emd'], new_embedding), axis=1)

# 유사한 플리 2개 id 추출
sim_ply_id = list(ply_title_embedding.sort_values(by='cos_sim', ascending=False)['ply_id'][:5])

In [345]:
sim_ply_id

[100425, 13343, 37504, 114848, 126736]

In [320]:
sql = 'SELECT * FROM ply_meta WHERE ply_id = 100425 or ply_id = 13343 or ply_id = 37504'
sim_ply_meta = pd.read_sql(sql, conn)
sim_ply_meta.head()

C:\Users\k9942\AppData\Local\Temp\ipykernel_23216\2459628092.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  sim_ply_meta = pd.read_sql(sql, conn)


,ply_id,ply_title,ply_tag_lst,ply_like_cnt,ply_updt
0,13343,비오는 오후 카페에서 사람 구경,"비오는날, 혼자있고싶을때, 카페음악, 인디음악",8,2017-04-05 18:59:46.000
1,37504,비오는 날 창가에 앉아서,"비오는날, 까페",9,2015-12-02 11:21:17.000
2,100425,비오는날 창밖을 보며,"커피, 감성, 기분전환, 사랑, 잔잔한, 뉴에이지, 겨울비, 봄비, 비오는날, 카페",89,2018-02-28 13:05:16.000


In [ ]:
[234462, 451946, 678145, 30748, 462092, 658645, 253755, 476469, 506254, 61029]

In [252]:
sm = 'SELECT * FROM song_meta WHERE song_id = 234462 or song_id = 451946 or song_id = 678145'
sm = pd.read_sql(sm, conn)
sm.head()

C:\Users\k9942\AppData\Local\Temp\ipykernel_23216\1235305545.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  sm = pd.read_sql(sm, conn)


,song_id,song_name,artist_name_lst,song_gnr_lst
0,234462,오늘 서울은 하루종일 맑음 (Vocal 윤하),"토이, 윤하 (YOUNHA)",발라드
1,451946,바래진 기억에,박지윤,발라드
2,678145,Goodbye,라디 (Ra. D),R&B/Soul


---

In [5]:
# # 플리 id 마지막 값 가져오기
# cur = conn.cursor()
# cur.execute('select ply_id from ply_meta order by ply_id desc limit 1;')
# last_ply_id = cur.fetchall()
# new_ply_id = last_ply_id[0][0] + 1

In [6]:
# new_ply_id

153429

In [6]:
# Insert new ply_title 
# cur.execute(f'INSERT INTO ply_meta (ply_id, ply_title, ply_updt) VALUES ({new_ply_id}, "{new_title}", now());')
# conn.commit()

In [7]:
# # input_title embedding 값 만들기
# new_embedding = model.encode(new_title)

In [ ]:
# new_embedding

In [8]:
# Insert new title embedding
# cur.execute(f'INSERT INTO ply_title_embedding (ply_id, ply_title_emd) VALUES ({new_ply_id}, "{new_embedding}");')
# conn.commit()

In [9]:
# 가장 유사한 플레이리스트 추출
def cos_sim(A, B):
  return dot(A, B)/(norm(A)*norm(B))

In [10]:
sql = 'SELECT * FROM ply_title_embedding'
ply_title_embedding = pd.read_sql(sql, conn)

C:\Users\k9942\AppData\Local\Temp\ipykernel_5984\837042529.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  ply_title_embedding = pd.read_sql(sql, conn)


In [12]:
ply_title_embedding.head()

,emd_id,ply_id,ply_title_emd
0,1,117162,"-0.6645728946, -0.7428655624, 0.9763435721, -0..."
1,2,151638,"-0.0026101221, 0.1385624856, 1.9987051487, 0.0..."
2,3,27984,"-0.4456211329, 0.0304336753, 1.2807025909, -0...."
3,4,30079,"-0.1292974651, 0.07028008250000001, 1.06260252..."
4,5,52572,"-0.21672616900000002, -0.0234135706, 1.0831099..."


In [16]:
# str -> embedding(array)
np.fromstring(ply_title_embedding['ply_title_emd'][0], dtype=float, sep=',')

array([-0.66457289, -0.74286556,  0.97634357, -0.02748305,  0.60607541,
       -0.55852813, -0.47341421,  0.07493094,  0.0254955 , -0.62810802,
        0.18689969,  0.0879852 , -0.11623359,  0.43376115,  0.03830802,
        0.08585956,  0.39989254, -0.05319985,  0.35957113, -0.75158173,
        0.53430384,  0.13137089,  0.17460242, -0.27146977, -0.23704727,
       -0.8135041 ,  0.06387217, -0.05302037,  0.93709463,  0.35246709,
       -0.30167225,  0.72681719, -0.44047704, -0.37019578, -0.25567219,
       -0.2047333 , -0.20838092,  0.80500418,  0.09485076, -0.07501914,
       -0.01121316, -0.43217891,  0.19892727, -0.0323044 , -0.33149099,
       -0.10192275,  0.08870346,  0.47512895,  0.26690188, -0.77355009,
       -0.14977601, -0.08149196,  0.99988759,  0.74196529, -0.71967936,
        0.14848152, -0.08216969,  0.5500561 ,  0.28603575,  0.0819269 ,
       -0.66555899,  0.16254689,  0.07249577,  0.15712616, -0.31431955,
        0.13759488,  1.25317848, -0.0888943 ,  0.17873168,  0.68

In [17]:
sql = 'SELECT * FROM ply_title_embedding'
ply_title_embedding = pd.read_sql(sql, conn)

# str -> embedding(array)
ply_title_embedding['ply_title_emd'] = ply_title_embedding.apply(lambda x: np.fromstring(x['ply_title_emd'],dtype=float, sep=','), axis=1)

# cosine similarity 계산
ply_title_embedding['cos_sim'] = ply_title_embedding.apply(lambda x: cos_sim(x['ply_title_emd'], new_embedding), axis=1)

# 유사한 플리 2개 id 추출
sim_ply_id = list(ply_title_embedding.sort_values(by='cos_sim', ascending=False)['ply_id'][:2])

C:\Users\k9942\AppData\Local\Temp\ipykernel_5984\2113611638.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  ply_title_embedding = pd.read_sql(sql, conn)


In [18]:
sim_ply_id

[50675, 37349]

In [22]:
sql = 'SELECT * FROM ply_meta WHERE ply_id = 50675 or ply_id = 37349'
sim_ply_meta = pd.read_sql(sql, conn)


C:\Users\k9942\AppData\Local\Temp\ipykernel_5984\811671671.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  sim_ply_meta = pd.read_sql(sql, conn)


In [23]:
sim_ply_meta

,ply_id,ply_title,ply_tag_lst,ply_like_cnt,ply_updt
0,37349,크리스마스 시즌에 듣기 좋은 노래,"사랑, 겨울, 설렘",21,2016-12-01 18:37:47.000
1,50675,크리스마스에 적합한 신나는 노래들,댄스,24,2012-12-20 09:15:30.000


In [7]:
conn.commit()

In [8]:
conn.close()